In [1]:
import pandas as pd
import numpy as np
from math import ceil
from collections import defaultdict
import yaml
import base64
import requests
import json
import time

In [2]:
class DataQueryInterface(object):

    def __init__(self, username: str, password: str,
                 crt: str = "api_macrosynergy_com.crt",
                 key: str = "api_macrosynergy_com.key"):

        self.auth = base64.b64encode(bytes(f'{username:s}:{password:s}',
                                           "utf-8")).decode('ascii')
        self.headers = {"Authorization": f"Basic {self.auth:s}"}
        self.base_url = "https://platform.jpmorgan.com/research/dataquery/api/v2"
        self.cert = (crt, key)

    def _fetch(self, endpoint: str = "/groups", select: str = "groups",
               params: dict = None):
        url = self.base_url + endpoint
        results = []

        with requests.get(url=url, cert=self.cert, headers=self.headers, params=params) \
                as r:
            self.last_response = r.text
        response = json.loads(self.last_response)

        assert select in response.keys()
        results.extend(response[select])

        return results

    def _fetch_ts(self, endpoint: str, params: dict, start_date: str = None,
                  end_date: str = None, calendar: str = "CAL_ALLDAYS",
                  frequency: str = "FREQ_DAY"):

        params["format"] = "JSON"
        params["start-date"] = start_date
        params["end-date"] = end_date
        params["calendar"] = calendar
        params["frequency"] = frequency
        results = self._fetch(endpoint=endpoint, params=params, select="instruments")

        return results

    def get_tickers(self, tickers, original_metrics, **kwargs):

        no_tickers = len(tickers)
        iterations = ceil(no_tickers / 20)
        remainder = no_tickers % 20

        results = []
        tickers_copy = tickers.copy()
        for i in range(iterations):
            if i < (iterations - 1):
                tickers = tickers_copy[i * 20: (i * 20) + 20]
            else:
                tickers = tickers_copy[-remainder:]

            params = {"expressions": tickers}
            output = self._fetch_ts(endpoint="/expressions/time-series",
                                    params=params, **kwargs)
            results.extend(output)

        no_metrics = len(set([tick.split(',')[-1][:-1] for tick in tickers_copy]))
        print(f"Number of metrics: {no_metrics}.")

        results_dict = self.isolate_timeseries(results, original_metrics)
        results_dict = self.valid_ticker(results_dict)

        results_copy = results_dict.copy()
        try:
            results_copy.popitem()
        except Exception as err:
            print(err)
            print("None of the tickers are available in the Database.")
            return
        else:
            return self.dataframe_wrapper(results_dict, no_metrics, original_metrics)

    @staticmethod
    def isolate_timeseries(list_, metrics):
        output_dict = defaultdict(dict)
        size = len(list_)

        for i in range(size):
            try:
                r = list_.pop()
            except IndexError:
                break
            else:
                dictionary = r['attributes'][0]
                ticker = dictionary['expression'].split(',')
                metric = ticker[-1][:-1]

                ticker_split = ','.join(ticker[:-1])
                ts_arr = np.array(dictionary['time-series'])

                if ticker_split not in output_dict:
                    output_dict[ticker_split]['real_date'] = ts_arr[:, 0]
                    output_dict[ticker_split][metric] = ts_arr[:, 1]
                else:
                    output_dict[ticker_split][metric] = ts_arr[:, 1]

        no_rows = ts_arr[:, 1].size
        modified_dict = {}
        d_frame_order = ['real_date'] + metrics

        for k, v in output_dict.items():
            arr = np.empty(shape=(no_rows, len(d_frame_order)), dtype=object)
            for i, metric in enumerate(d_frame_order):
                arr[:, i] = v[metric]

            modified_dict[k] = arr
        return modified_dict

    @staticmethod
    def column_check(v, col):
        returns = list(v[:, col])
        condition = all([isinstance(elem, type(None)) for elem in returns])

        return condition

    def valid_ticker(self, _dict):
        dict_copy = _dict.copy()
        for k, v in _dict.items():

            condition = self.column_check(v, 1)
            if condition:
                print(f"The ticker, {k}, does not exist in the Database.")
                dict_copy.pop(k)
            else:
                continue

        return dict_copy

    @staticmethod
    def dataframe_wrapper(_dict, no_metrics, original_metrics):
        tickers_no = len(_dict.keys())
        length = list(_dict.values())[0].shape[0]

        arr = np.empty(shape=(length * tickers_no, 3 + no_metrics), dtype=object)
        print(arr.shape)

        i = 0
        for k, v in _dict.items():
            ticker = k.split(',')
            ticker = ticker[1].split('_')

            cid = ticker[0]
            xcat = '_'.join(ticker[1:])

            cid_broad = np.repeat(cid, repeats=v.shape[0])
            xcat_broad = np.repeat(xcat, repeats=v.shape[0])
            data = np.column_stack((cid_broad, xcat_broad, v))

            row = i * v.shape[0]
            arr[row:row + v.shape[0], :] = data
            i += 1

        columns = ['cid', 'xcat', 'real_date']
        cols_output = columns + original_metrics

        df = pd.DataFrame(data=arr, columns=cols_output)

        df['real_date'] = pd.to_datetime(df['real_date'], yearfirst=True)
        df = df[df['real_date'].dt.dayofweek < 5]

        return df.fillna(value=np.nan).reset_index(drop=True)

In [3]:
def dq_dataframe(tickers, metrics_=["value"], start_d="2000-01-01"):

    unique_tickers = list(set(tickers))
    dq_tickers = []
    for metric in metrics_:
        dq_tickers = dq_tickers + ["DB(JPMAQS," + tick + f",{metric})"
                                   for tick in unique_tickers]
    print(f"tickers: {tickers}")

    with open("config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    dq = DataQueryInterface(username=cf["dq"]["username"], password=cf["dq"]["password"],
                            crt="api_macrosynergy_com.crt",
                            key="api_macrosynergy_com.key")

    df_ts = dq.get_tickers(tickers=dq_tickers, original_metrics=metrics,
                                 start_date=start_d)

    if isinstance(df_ts, pd.DataFrame):
        df_ts = df_ts.sort_values(['cid', 'xcat', 'real_date']).reset_index(drop=True)

        return df_ts

In [4]:
cids_dmca = ['AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'JPY', 'NOK', 'NZD', 'SEK', 'USD']  # DM currency areas
cids_dmec = ['DEM', 'ESP', 'FRF', 'ITL', 'NLG']  # DM euro area countries
cids_latm = ['ARS', 'BRL', 'COP', 'CLP', 'MXN', 'PEN']  # Latam countries
cids_emea = ['HUF', 'ILS', 'PLN', 'RON', 'RUB', 'TRY', 'ZAR']  # EMEA countries
cids_emas = ['CNY', 'HKD', 'IDR', 'INR', 'KRW', 'MYR', 'PHP', 'SGD', 'THB', 'TWD']  # EM Asia countries
cids_dm = cids_dmca + cids_dmec
cids_em = cids_latm + cids_emea + cids_emas
cids = sorted(cids_dm + cids_em)

cids_eufx = ['CHF', 'HUF', 'NOK', 'PLN', 'RON', 'SEK']  # EUR benchmark
cids_g2fx = ['GBP', 'RUB', 'TRY']  # dual benchmark
cids_usfx = ['AUD', 'BRL', 'CAD', 'CLP', 'CNY', 'COP', 'EUR', 'IDR', 'ILS', 'INR', 'JPY', 'KRW', 'MYR',
             'MXN', 'NZD', 'PEN', 'PHP', 'SGD', 'THB', 'TWD', 'ZAR']  # USD benchmark
cids_fx = cids_usfx + cids_eufx + cids_g2fx

metrics = ['value', 'eop_lag']

cids = cids_dmca + cids_emea
cids += cids_emas
cids += cids_usfx
cids = list(set(cids))

print(f"Length of cross-sectional list: {len(cids)}")

metrics = ['value', 'eop_lag']
tix_1 = [cid + '_CPIXFE_SJA_P6M6ML6AR' for cid in cids]

start = time.time()
df = dq_dataframe(tix_1, metrics, "2011-01-01")
end = time.time() - start
print(f"Time taken: {end}.")
print(df)

Length of cross-sectional list: 32
tickers: ['INR_CPIXFE_SJA_P6M6ML6AR', 'EUR_CPIXFE_SJA_P6M6ML6AR', 'GBP_CPIXFE_SJA_P6M6ML6AR', 'PEN_CPIXFE_SJA_P6M6ML6AR', 'CNY_CPIXFE_SJA_P6M6ML6AR', 'AUD_CPIXFE_SJA_P6M6ML6AR', 'KRW_CPIXFE_SJA_P6M6ML6AR', 'MYR_CPIXFE_SJA_P6M6ML6AR', 'ILS_CPIXFE_SJA_P6M6ML6AR', 'ZAR_CPIXFE_SJA_P6M6ML6AR', 'IDR_CPIXFE_SJA_P6M6ML6AR', 'SGD_CPIXFE_SJA_P6M6ML6AR', 'TRY_CPIXFE_SJA_P6M6ML6AR', 'RON_CPIXFE_SJA_P6M6ML6AR', 'NZD_CPIXFE_SJA_P6M6ML6AR', 'SEK_CPIXFE_SJA_P6M6ML6AR', 'USD_CPIXFE_SJA_P6M6ML6AR', 'RUB_CPIXFE_SJA_P6M6ML6AR', 'NOK_CPIXFE_SJA_P6M6ML6AR', 'PLN_CPIXFE_SJA_P6M6ML6AR', 'COP_CPIXFE_SJA_P6M6ML6AR', 'JPY_CPIXFE_SJA_P6M6ML6AR', 'THB_CPIXFE_SJA_P6M6ML6AR', 'TWD_CPIXFE_SJA_P6M6ML6AR', 'CHF_CPIXFE_SJA_P6M6ML6AR', 'MXN_CPIXFE_SJA_P6M6ML6AR', 'HKD_CPIXFE_SJA_P6M6ML6AR', 'PHP_CPIXFE_SJA_P6M6ML6AR', 'BRL_CPIXFE_SJA_P6M6ML6AR', 'CLP_CPIXFE_SJA_P6M6ML6AR', 'CAD_CPIXFE_SJA_P6M6ML6AR', 'HUF_CPIXFE_SJA_P6M6ML6AR']
Number of metrics: 2.
The ticker, DB(JPMAQS,HKD_CPIXFE_SJA_